# Imports

In [ ]:
#standard imports
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
import numpy as np

#GUI imports
from tkinter import *
from tkinter.filedialog import askopenfilename
from tkinter import ttk
from PIL import Image, ImageTk
import tkinter.simpledialog
from tkinter.messagebox import showinfo

# Data

In [ ]:
#read in data as csv file
#year 2013
joined13 = pd.read_csv("Data/joined_shots_2013.csv")

#year 2014
joined14 = pd.read_csv("Data/joined_shots_2014.csv")

#put both joined csv files together
joined = pd.concat([joined13, joined14], ignore_index = True)

In [ ]:
#all available columns
joined.columns

In [ ]:
#number of different players in the dataset
joined.PLAYER_ID.nunique()

In [ ]:
joined.head(10)

## Visualizations

In [ ]:
#visualize distributions of shot distance (when shot was successful)
#and distance of the closest defender (when shot was successful)
figure, axis = plt.subplots(2, 4)
figure.tight_layout()

axis[0, 0].boxplot(joined["SHOT_DISTANCE"])
axis[0, 0].set_title("Boxplot of shot distance")

axis[1, 0].hist(joined["SHOT_DISTANCE"])
axis[1, 0].set_title("Histogram of shot distance")

axis[0, 1].boxplot(joined[joined["SHOT_MADE_FLAG"] == 1]["SHOT_DISTANCE"])
axis[0, 1].set_title("Boxplot of distance of made shots")

axis[1, 1].hist(joined[joined["SHOT_MADE_FLAG"] == 1]["SHOT_DISTANCE"])
axis[1, 1].set_title("Histogram of distance of made shots")

axis[0, 2].boxplot(joined["CLOSE_DEF_DIST"])
axis[0, 2].set_title("Boxplot of distance of closest defender")

axis[1, 2].hist(joined["CLOSE_DEF_DIST"])
axis[1, 2].set_title("Histogram of distance of closest defender")

axis[0, 3].boxplot(joined[joined["SHOT_MADE_FLAG"] == 1]["CLOSE_DEF_DIST"])
axis[0, 3].set_title("Boxplot of distance of closest defender when shot was made")

axis[1, 3].hist(joined[joined["SHOT_MADE_FLAG"] == 1]["CLOSE_DEF_DIST"])
axis[1, 3].set_title("Histogram of distance of closest defender when shot was made")

## Player dictionary

In [ ]:
#create a dict that maps the player to his correct ID
player_dict = {}

for i in range(len(joined)):
    if joined["PLAYER_ID"][i] in player_dict:
        pass
    else:
        player_dict[joined["PLAYER_ID"][i]] = joined["PLAYER_NAME"][i]

In [ ]:
key_list = list(player_dict.keys())
val_list = list(player_dict.values())

def player_id(player):
    """
    Returns the correct ID for the chosen player.

            Parameters:
                    player (str)    : name of the player

            Returns:
                    player_id (int) : ID of the player
    """
    
    position = val_list.index(player)
    player_id = key_list[position]
    return player_id

# Regression

## Simple Linear Regression

In [ ]:
x = joined[["SHOT_DISTANCE", "PLAYER_ID"]]
y = joined["SHOT_MADE_FLAG"]

In [ ]:
#create a simple linear regression model and show its summary
X = sm.add_constant(x)
results = sm.OLS(y, X).fit()
results.summary()

## Random Forest

In [ ]:
X_rf = joined[["SHOT_DISTANCE", "PLAYER_ID"]]
y_rf = joined["SHOT_MADE_FLAG"]

#create a random forest model and print its r-squared value
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
randFor = regressor.fit(X_rf, y_rf)
randFor.score(X_rf, y_rf)

In [ ]:
def shot_prediction(player, shot_distance):
    """
    Predicts the probability of a successful shot.

            Parameters:
                    player (str)          : name of the player
                    shot_distance (int)   : distance of the shot in ft

            Returns:
                    pred_percentage (int) : probability of a successful shot
    """
    
    prediction = regressor.predict([[shot_distance, player_id(player)]])[0]
    pred_percentage = round(prediction, 3) * 100
    return pred_percentage

In [ ]:
shot_prediction("Kobe Bryant", 15)

# GUI

## Design of the GUI

In [ ]:
def pred_on_ok():
    """
    Uses the entered player name and entered distance to predict and displays
    the probability of the shot going in.

            Parameters:
                    None

            Returns:
                    None
    """
    
    name = entry_player.get()
    distance = entry_shot.get()
    prediction = shot_prediction(name, distance)
    showinfo(title='Information',
            message="When the player {} takes a shot with a distance of {} ft from the basket," \
                     "the estimated probability of scoring is {} %".format(name, distance, prediction))


def getorigin_shot(eventorigin):
    """
    Creates a red circle at the position of the mouse click for the shooter.

            Parameters:
                    eventorigin (event) : position of mouse click by the user

            Returns:
                    None
    """
    
    global x0,y0
    x0 = eventorigin.x
    y0 = eventorigin.y
    print(x0,y0)
    if x0 >= 290 and x0 <= 700 and y0 >= 140 and y0 <= 450:
        w.create_oval(x0, y0, x0+10, y0+10, fill="red")
    #w.bind("<Button 1>", printcoords)

    
# def getorigin_def(eventorigin):
#    """
#    Creates a blue circle at the position of the mouse click for the defender.

#            Parameters:
#                    eventorigin (event) : position of mouse click by the user

#            Returns:
#                    None
#    """
   
#    global x1,y1
#    x1 = eventorigin.x
#    y1 = eventorigin.y
#    print(x1,y1)
#    w.create_oval(x1, y1, x1+10, y1+10, fill="blue")
#    #w.bind("<Button 1>", printcoords)
# #mouseclick event
# w.bind("<Button 1>", getorigin_def)


# #coordinate transformation into distance from bucket
# def printcoords(event):
#    xmpx = xe-x0
#    xm = xmt/xmpx
#    ympx = ye-y0
#    ym = -ymp/ympx

#    #coordinate transformation
#    newx = (event.x-x0)*(xm)+xc
#    newy = (event.y-y0)*(ym)+yc

#    #outputting x and y coords to console
#    print (newx,newy)


#create a window with a title and two labels, both above a text field, where the user can enter the player
#and the shot distance for the prediction
root = Tk()
root.title("Player and shot selection")

Label(text='Which player? (ex: Kobe Bryant, Derrick Rose)').pack(side=TOP,padx=10,pady=10)

entry_player = Entry(root, width=10)
entry_player.pack(side=TOP,padx=10,pady=10)

Label(text='Which distance in ft? (ex: 5, 12)').pack(side=TOP,padx=20,pady=20)

entry_shot = Entry(root, width=10)
entry_shot.pack(side=TOP,padx=20,pady=20)


#create OK button to print the prediction when pressed
Button(root, text='OK', command=pred_on_ok).pack(side=LEFT)


#setting up a tkinter canvas
w = Canvas(root, width=1000, height=1000)
w.pack()

#adding image of basketball court
image = Image.open("pict--half-basketball-court-basketball-courts-vector-stencils-library.png")
resize_image = image.resize((800, 400))
img = ImageTk.PhotoImage(resize_image)

w.create_image((500, 300), image = img)


#mouseclick event
w.bind("<Button 1>", getorigin_shot)


root.mainloop()